In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# 학습 함수 정의
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    train_loss = running_loss / len(dataloader.dataset)
    return train_loss

# 평가 함수 정의
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    
    test_loss = running_loss / len(dataloader.dataset)
    test_accuracy = correct / len(dataloader.dataset)
    return test_loss, test_accuracy

In [3]:
# 데이터 전처리를 위한 변환 함수 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 이미지 정규화
])

# 데이터셋 로드
train_dataset = torchvision.datasets.ImageFolder(root='../data/Pneumonia_united/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='../data/Pneumonia_united/test', transform=transform)

# 데이터로더 생성
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
test_accuracy = 0
while test_accuracy < 0.78:
    # MobileNet 모델 불러오기
    model = models.mobilenet_v2(pretrained=True)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, 2)  # 분류할 클래스 수에 맞게 마지막 레이어 수정

    # 모델 학습 설정
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)
    criterion = criterion.to(device)

    # 학습 및 평가 실행
    num_epochs = 1
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer)
        test_loss, test_accuracy = evaluate(model, test_loader, criterion)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

        if test_accuracy >= 0.78:
            break

Epoch [1/1], Train Loss: 0.2938, Test Loss: 0.6332, Test Accuracy: 0.7812


In [6]:
import torch
from sklearn.metrics import confusion_matrix

def calculate_confusion_matrix(model, dataloader, device):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    confusion_mat = confusion_matrix(all_labels, all_predictions)
    return confusion_mat

In [7]:
calculate_confusion_matrix(model, test_loader, device)

array([[54, 26],
       [ 9, 71]], dtype=int64)

In [8]:
torch.save(model, 'Pneumonia3.h5')